## Using Twitter API to get Tweets with Specific days



## Create the Keywords and Hashtags list


In [ ]:
hashtags = [
           'الصين', 'كورونا','فيروس-كورونا','كورونا-الجديد','كوفيد','فيروس-كورونا-المستجد','كلنا-مسؤول','ابقوا-في-منازلكم','فعاليات-الحجر','الحجر-المنزلي','حظر-تجول','خليك-في-البيت',
            "خلك_في_البيت", "خليك_في_البيت","خليك_في_بيتك", "خلك_في_بيتك"
]

keywords = ["كورونا", "كوفيد","كمامة", "معقم", "(غسل AND اليدين)", "(حظر AND التجوال) ", "(الحجر AND الصحي) ", "(العزل AND المنزلي) " ,"(التباعد AND الاجتماعي)"]

"""
Join the hastags and keywords together
"""

composed_hashtags = ''
for i in range(len(hashtags)):
  if(i < len(hashtags)-1):
    composed_hashtags += '#' + hashtags[i] + ' OR '
  else:
    composed_hashtags += '#' + hashtags[i] 
composed_hashtags

composed_keywords = ''

for i in range(len(keywords)):
  if(i < len(keywords)-1):
    composed_keywords += keywords[i] + ' OR '
  else:
    composed_keywords += keywords[i] 
composed_keywords

composed_query = composed_keywords +' OR ' + composed_hashtags
composed_query



'كورونا OR كوفيد OR كمامة OR معقم OR (غسل AND اليدين) OR (حظر AND التجوال)  OR (الحجر AND الصحي)  OR (العزل AND المنزلي)  OR (التباعد AND الاجتماعي) OR #الصين OR #كورونا OR #فيروس-كورونا OR #كورونا-الجديد OR #كوفيد OR #فيروس-كورونا-المستجد OR #كلنا-مسؤول OR #ابقوا-في-منازلكم OR #فعاليات-الحجر OR #الحجر-المنزلي OR #حظر-تجول OR #خليك-في-البيت OR #خلك_في_البيت OR #خليك_في_البيت OR #خليك_في_بيتك OR #خلك_في_بيتك'

## Import required labraries

In [ ]:
import tweepy
import webbrowser
import time
import json
import csv
import tweepy
import re
import pandas as pd
import datetime
import sys

## How to get Access keys to use Twitter API

In [1]:
"""
To access this keys you want to go throw the following steps

1. Create Twitter acount if you do not have one
2. Apply for developer acount and wait for agreement
3. Login to Twitter with your acount, create a new application and create Your Access Token.

"""
consumer_key    = "xxxxxxxx"
consumer_secret = "xxxxxxxxx"
callback_url = "xxxxxxxxxx"
auth = tweepy.OAuthHandler(consumer_key , consumer_secret,callback_url)
redirect_link = auth.get_authorization_url()
print(redirect_link)
code_sent = input('what is the code')
auth.get_access_token(code_sent)

In [ ]:
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)

api = tweepy.API(auth, wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

## Get Tweets for one day

In [ ]:
"""
INPUTS:
    consumer_key, consumer_secret, access_token, access_token_secret: codes 
    telling twitter that we are authorized to access this data
    hashtag_phrase: the combination of hashtags to search for
OUTPUTS:
    none, simply save the tweet info to a spreadsheet

    This code is to get tweets for one day: "2020-09-06" to until="2020-09-07"
"""
from retrying import retry
@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000)
def search_for_hashtags(api, hashtag_phrase):
   try:
      for tweet in tweepy.Cursor(api.search, q= hashtag_phrase + "-filter:retweets", lang="ar",since="2020-09-06", until="2020-09-07", tweet_mode='extended').items():
         #w.writerow([tweet.created_at, tweet.full_text.replace('\n',' '), tweet.truncated ,tweet.id ,tweet.user.screen_name.encode('utf-8'),tweet.user.name.encode('utf-8'), tweet.user.id,tweet.user.location, tweet.user.friends_count,tweet.user.followers_count, tweet.user.statuses_count ,[e['text'] for e in tweet._json['entities']['hashtags']], tweet.coordinates, tweet.is_quote_status, tweet.quoted_status.replace('\n',' '), tweet.quote_count, tweet.reply_count, tweet.retweet_count, tweet.favorite_count, tweet.retweet_status])
         all_hash = tweet.entities['hashtags']
         with open("covid_data.csv", "a", encoding='utf-8') as f:
            f.write("%s,%s,%d,%s,%d,%s,%d,%d,%d,%d,%d,%s\n" % (tweet.created_at, tweet.full_text.replace('\n',' ').replace(',',' ') ,tweet.id ,tweet.user.screen_name,tweet.user.id,tweet.user.location.replace('\n',' ').replace(',',' '), tweet.user.friends_count, tweet.user.followers_count, tweet.user.statuses_count, tweet.retweet_count, tweet.favorite_count, zip(all_hash)))
         print(tweet.created_at)

   except tweepy.TweepError as e:
      # Just exit if any error
      print("some error : " + str(e))
      raise
    

In [2]:


if __name__ == '__main__':
    filename = "covid_data.csv"
    with open("covid_data.csv", "w", encoding='utf-8') as f:
       f.write("timestamp, tweet_text, tweet_id,username, user_id,user_location, user_friends_count, user_followers_count, user_statuses_count,tweet_retweet_count, tweet_favorite_count, all_hashtags\n")
    search_for_hashtags(api, composed_query)


In [ ]:
# for test only



# if __name__ == '__main__':
#          filename = "data.csv"
#          with open("data.csv", "w", encoding='utf-8') as f:
#             f.write("timestamp, tweet_text, tweet_id, user_name, user_id,user_location, user_friends_count, user_followers_count, user_statuses_count,tweet_retweet_count, tweet_favorite_count, all_hashtags\n")
#             #f.write("time\n")
#          for tweet in tweepy.Cursor(api.search, q="كورونا"+"-filter:retweets", lang="ar", tweet_mode='extended').items():
#              hash = tweet.entities['hashtags']
#              #hash = tweet.entities['hashtags'].join("-")
#              #hash=(" ".join(hash))
             
#              with open("data.csv", "a", encoding='utf-8') as f:
#                 f.write("%s,%s,%d,%s,%d,%s,%d,%d,%d,%d,%d,%s\n" % (tweet.created_at, tweet.full_text.replace('\n',' ').replace(',',' ') ,tweet.id ,tweet.user.screen_name,tweet.user.id,tweet.user.location.replace('\n',' ').replace(',',' '), tweet.user.friends_count, tweet.user.followers_count, tweet.user.statuses_count, tweet.retweet_count, tweet.favorite_count, zip(hash) ))#% (tweet.created_at, tweet.full_text.replace('\n',' ') ,tweet.id ,tweet.user.screen_name.replace('\n',' '), tweet.user.id,tweet.user.location.replace('\n',' '), tweet.user.friends_count,tweet.user.followers_count, tweet.user.statuses_count ,tweet.id_str,tweet.id_str,tweet.retweet_count, tweet.favorite_count))
#              print(tweet.created_at)

## Read the dataset

In [3]:
import pandas as pd
#result = pd.read_csv('covid_data.csv')
#result = pd.read_csv('covid_data.csv', lineterminator='\n')

df = pd.read_csv('covid_data.csv', lineterminator='\n', error_bad_lines=False)
df

## Save the data to your Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df.to_csv("drive/My Drive/Covid_Data/covid_data_6_9.csv")